In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("24121003_yellowtaxi_trip_count").getOrCreate()

24/12/12 08:54:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
import os
trip_files = '/trips/*'
zone_file = 'taxi+_zone_lookup.csv'
directory = os.path.join(os.getcwd(), 'data')

In [3]:
trips_df = spark.read.csv(f'file:///{directory}/{trip_files}', inferSchema=True, header=True)

In [4]:
zone_df = spark.read.csv(f'file:///{directory}/{zone_file}', inferSchema=True, header=True)

In [5]:
trips_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [6]:
zone_df.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [7]:
trips_df.createOrReplaceTempView('trips')
zone_df.createOrReplaceTempView('zone')

In [8]:
query = '''
select
t.VendorID,
TO_DATE(t.tpep_pickup_datetime) as pickup_date,
TO_DATE(t.tpep_dropoff_datetime) as dropoff_date,
HOUR(t.tpep_pickup_datetime)  as pickup_time,
HOUR(t.tpep_dropoff_datetime) as dropoff_time,
t.passenger_count,
t.trip_distance,
t.tip_amount,
t.total_amount,
t.payment_type,
pz.Zone as pickup_zone,
dz.Zone as dropoff_zone
from trips t
LEFT JOIN zone pz ON t.PULocationID = pz.LocationID
LEFT JOIN zone dz ON t.DOLocationID = dz.LocationID
'''

In [9]:
comb_df = spark.sql(query)

In [10]:
comb_df.count()

15000700

In [11]:
comb_df.show(5)

+--------+-----------+------------+-----------+------------+---------------+-------------+----------+------------+------------+-----------------+--------------+
|VendorID|pickup_date|dropoff_date|pickup_time|dropoff_time|passenger_count|trip_distance|tip_amount|total_amount|payment_type|      pickup_zone|  dropoff_zone|
+--------+-----------+------------+-----------+------------+---------------+-------------+----------+------------+------------+-----------------+--------------+
|       2| 2021-03-01|  2021-03-01|          0|           0|              1|          0.0|       0.0|         4.3|           2|               NV|            NV|
|       2| 2021-03-01|  2021-03-01|          0|           0|              1|          0.0|       0.0|         3.8|           2|   Manhattanville|Manhattanville|
|       2| 2021-03-01|  2021-03-01|          0|           0|              1|          0.0|       0.0|         4.8|           2|   Manhattanville|Manhattanville|
|       1| 2021-03-01|  2021-03-01

In [12]:
comb_df.createOrReplaceTempView('comb')

In [13]:
query = '''
select pickup_date, pickup_time 
from comb 
where pickup_time>0
'''

In [21]:
spark.sql(query).show()

+-----------+-----------+
|pickup_date|pickup_time|
+-----------+-----------+
| 2021-02-28|         23|
| 2021-02-28|         23|
| 2021-02-28|         23|
| 2021-02-28|         23|
| 2021-02-28|         23|
| 2021-02-28|         23|
| 2021-02-28|         23|
| 2021-03-01|         22|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
+-----------+-----------+
only showing top 20 rows



In [22]:
query = '''
select pickup_date, pickup_time 
from comb 
where pickup_date<'2020-12-31'
'''
spark.sql(query).show()

[Stage 16:=============================>                            (1 + 1) / 2]

+-----------+-----------+
|pickup_date|pickup_time|
+-----------+-----------+
| 2009-01-01|          0|
| 2008-12-31|         23|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          1|
| 2009-01-01|          0|
| 2008-12-31|         23|
| 2008-12-31|         23|
| 2008-12-31|         23|
| 2008-12-31|         23|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|         16|
| 2009-01-01|         16|
| 2009-01-01|          0|
| 2009-01-01|          0|
+-----------+-----------+
only showing top 20 rows



In [17]:
comb_df.describe()

DataFrame[summary: string, VendorID: string, pickup_time: string, dropoff_time: string, passenger_count: string, trip_distance: string, tip_amount: string, total_amount: string, payment_type: string, pickup_zone: string, dropoff_zone: string]

In [18]:
spark.sql(query).explain()

== Physical Plan ==
*(3) Project [cast(tpep_pickup_datetime#17 as date) AS pickup_date#76, hour(cast(tpep_pickup_datetime#17 as timestamp), Some(Asia/Seoul)) AS pickup_time#78]
+- *(3) BroadcastHashJoin [DOLocationID#24], [LocationID#82], LeftOuter, BuildRight, false
   :- *(3) Project [tpep_pickup_datetime#17, DOLocationID#24]
   :  +- *(3) BroadcastHashJoin [PULocationID#23], [LocationID#68], LeftOuter, BuildRight, false
   :     :- *(3) Filter (isnotnull(tpep_pickup_datetime#17) AND (cast(tpep_pickup_datetime#17 as date) < 18627))
   :     :  +- FileScan csv [tpep_pickup_datetime#17,PULocationID#23,DOLocationID#24] Batched: false, DataFilters: [isnotnull(tpep_pickup_datetime#17), (cast(tpep_pickup_datetime#17 as date) < 18627)], Format: CSV, Location: InMemoryFileIndex[file:/home/lab06/src/DA-learning-course/Spark/data/trips/yellow_tripdata_2021-0..., PartitionFilters: [], PushedFilters: [IsNotNull(tpep_pickup_datetime)], ReadSchema: struct<tpep_pickup_datetime:string,PULocationID:i

In [19]:
#실행계획, 실행결과(4040)

query2 = '''
select pickup_date, pickup_time 
from comb 
where pickup_time > 0 and pickup_time<=12
'''
spark.sql(query2).show()

+-----------+-----------+
|pickup_date|pickup_time|
+-----------+-----------+
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
| 2021-03-01|          1|
+-----------+-----------+
only showing top 20 rows



In [22]:
query3 = '''
select pickup_date, pickup_time 
from comb 
where pickup_time > 0
group by pickup_date, pickup_time
order by pickup_date
'''
spark.sql(query3).show()

[Stage 22:=================================================>    (185 + 2) / 200]

+-----------+-----------+
|pickup_date|pickup_time|
+-----------+-----------+
| 2002-12-31|         23|
| 2003-01-05|          7|
| 2004-04-04|          4|
| 2008-12-31|         22|
| 2008-12-31|         23|
| 2009-01-01|          5|
| 2009-01-01|         20|
| 2009-01-01|         11|
| 2009-01-01|          1|
| 2009-01-01|         19|
| 2009-01-01|         17|
| 2009-01-01|         18|
| 2009-01-01|         23|
| 2009-01-01|          4|
| 2009-01-01|          3|
| 2009-01-01|         21|
| 2009-01-01|         12|
| 2009-01-01|         16|
| 2009-01-01|          2|
| 2009-01-01|         10|
+-----------+-----------+
only showing top 20 rows



# 실습 과제

In [14]:
from pyspark.sql.functions import *

# 데이터 전처리

In [43]:
# 음수 요금 값을 가진 레코드 필터링
negative_fare_df = trips_df.filter(col("total_amount") < 0).select("payment_type", "total_amount", "fare_amount", "tip_amount")

# 결과 출력
negative_fare_df.show()

+------------+------------+-----------+----------+
|payment_type|total_amount|fare_amount|tip_amount|
+------------+------------+-----------+----------+
|           3|       -15.3|      -15.0|       0.0|
|           4|        -6.3|       -2.5|       0.0|
|           4|       -11.3|       -7.5|       0.0|
|           2|       -30.3|      -26.5|       0.0|
|           3|        -6.8|       -3.0|       0.0|
|           4|       -11.3|       -7.5|       0.0|
|           3|        -6.3|       -2.5|       0.0|
|           4|        -3.8|       -2.5|       0.0|
|           3|        -6.3|       -2.5|       0.0|
|           4|       -35.8|      -32.0|       0.0|
|           4|        -6.8|       -3.5|       0.0|
|           3|        -6.8|       -3.5|       0.0|
|           4|        -5.8|       -2.5|       0.0|
|           4|       -14.3|      -11.0|       0.0|
|           2|       -12.3|       -9.0|       0.0|
|           4|       -14.3|      -11.0|       0.0|
|           4|        -5.8|    

In [15]:
from pyspark.sql.functions import col, when, avg, count, round, hour

# 1. 음수 요금 제거
trips_df = trips_df.filter(col("total_amount") >= 0)

# 2. 비정상적인 거리 데이터 제거
trips_df = trips_df.filter((col("trip_distance") >= 0) & (col("trip_distance") <= 100))

# 3. 요금이 비정상적으로 높은 경우 제거
trips_df = trips_df.filter((col("total_amount") / col("trip_distance")) <= 100)

# 4. 결측치 제거
trips_df = trips_df.na.drop()

# 5. 거리 구간화
trips_df = trips_df.withColumn(
    "distance_bin",
    when(col("trip_distance") < 3, "0-3 km")
    .when((col("trip_distance") >= 3) & (col("trip_distance") < 6), "3-6 km")
    .when((col("trip_distance") >= 6) & (col("trip_distance") < 9), "6-9 km")
    .when((col("trip_distance") >= 9) & (col("trip_distance") < 12), "9-12 km")
    .when((col("trip_distance") >= 12) & (col("trip_distance") < 15), "12-15 km")
    .when((col("trip_distance") >= 15) & (col("trip_distance") < 18), "15-18 km")
    .when((col("trip_distance") >= 18) & (col("trip_distance") < 21), "18-21 km")
    .when((col("trip_distance") >= 21) & (col("trip_distance") < 24), "21-24 km")
    .when((col("trip_distance") >= 24) & (col("trip_distance") < 27), "24-27 km")
    .when((col("trip_distance") >= 27) & (col("trip_distance") < 30), "27-30 km")
    .otherwise("30+ km")
)

# 6. 시간대 구분
trips_df = trips_df.withColumn("pickup_hour", hour(col("tpep_pickup_datetime")))

# View 생성 및 쿼리
trips_df.createOrReplaceTempView('trips')
zone_df.createOrReplaceTempView('zone')

query = '''
SELECT
  t.VendorID,
  TO_DATE(t.tpep_pickup_datetime) AS pickup_date,
  TO_DATE(t.tpep_dropoff_datetime) AS dropoff_date,
  HOUR(t.tpep_pickup_datetime) AS pickup_time,
  HOUR(t.tpep_dropoff_datetime) AS dropoff_time,
  t.passenger_count,
  t.trip_distance,
  t.tip_amount,
  t.total_amount,
  t.payment_type,
  pz.Zone AS pickup_zone,
  dz.Zone AS dropoff_zone,
  t.distance_bin
FROM trips t
LEFT JOIN zone pz ON t.PULocationID = pz.LocationID
LEFT JOIN zone dz ON t.DOLocationID = dz.LocationID
'''
comb_df = spark.sql(query)
comb_df.createOrReplaceTempView('comb')

## 1. 운행 거리와 요금의 상관관계 분석
> 쿼리작성, 데이터프레임으로도 구현

In [16]:
# 쿼리
distance_fare_query = '''
SELECT 
  distance_bin,
  ROUND(AVG(total_amount), 2) AS avg_fare,
  COUNT(total_amount) AS num_trips
FROM comb
GROUP BY distance_bin
ORDER BY 
  CASE 
    WHEN distance_bin = '0-3 km' THEN 1
    WHEN distance_bin = '3-6 km' THEN 2
    WHEN distance_bin = '6-9 km' THEN 3
    WHEN distance_bin = '9-12 km' THEN 4
    WHEN distance_bin = '12-15 km' THEN 5
    WHEN distance_bin = '15-18 km' THEN 6
    WHEN distance_bin = '18-21 km' THEN 7
    WHEN distance_bin = '21-24 km' THEN 8
    WHEN distance_bin = '24-27 km' THEN 9
    WHEN distance_bin = '27-30 km' THEN 10
    ELSE 11
  END;
'''
distance_fare_grouped_df = spark.sql(distance_fare_query)
distance_fare_grouped_df.show()

[Stage 11:======================================>               (144 + 2) / 200]

+------------+--------+---------+
|distance_bin|avg_fare|num_trips|
+------------+--------+---------+
|      0-3 km|   13.07| 10347228|
|      3-6 km|   22.56|  2172487|
|      6-9 km|   33.06|   558129|
|     9-12 km|   43.65|   319466|
|    12-15 km|   51.22|   120130|
|    15-18 km|   65.66|   150925|
|    18-21 km|   70.26|   161403|
|    21-24 km|    74.5|    34754|
|    24-27 km|   87.43|     9797|
|    27-30 km|   96.18|     5564|
|      30+ km|  129.01|    10866|
+------------+--------+---------+



In [17]:
# 데이터프레임

# 각 거리 구간별 평균 요금 계산 및 정렬
distance_fare_grouped_df = comb_df.groupBy("distance_bin").agg(
    round(avg("total_amount"), 2).alias("avg_fare"),
    count("total_amount").alias("num_trips")
).orderBy(
    when(col("distance_bin") == '0-3 km', 1)
    .when(col("distance_bin") == '3-6 km', 2)
    .when(col("distance_bin") == '6-9 km', 3)
    .when(col("distance_bin") == '9-12 km', 4)
    .when(col("distance_bin") == '12-15 km', 5)
    .when(col("distance_bin") == '15-18 km', 6)
    .when(col("distance_bin") == '18-21 km', 7)
    .when(col("distance_bin") == '21-24 km', 8)
    .when(col("distance_bin") == '24-27 km', 9)
    .when(col("distance_bin") == '27-30 km', 10)
    .otherwise(11)
)

# 결과 출력
distance_fare_grouped_df.show()


[Stage 14:==========================================>           (158 + 2) / 200]

+------------+--------+---------+
|distance_bin|avg_fare|num_trips|
+------------+--------+---------+
|      0-3 km|   13.07| 10347228|
|      3-6 km|   22.56|  2172487|
|      6-9 km|   33.06|   558129|
|     9-12 km|   43.65|   319466|
|    12-15 km|   51.22|   120130|
|    15-18 km|   65.66|   150925|
|    18-21 km|   70.26|   161403|
|    21-24 km|    74.5|    34754|
|    24-27 km|   87.43|     9797|
|    27-30 km|   96.18|     5564|
|      30+ km|  129.01|    10866|
+------------+--------+---------+



In [28]:
# 운행 거리와 요금의 상관 계수 계산
correlation = comb_df.select("trip_distance", "total_amount").stat.corr("trip_distance", "total_amount")

# 결과 출력
print(f"운행 거리와 요금의 상관 계수: {correlation:.2f}")

[Stage 54:==================================================>     (10 + 1) / 11]

운행 거리와 요금의 상관 계수: 0.93


> 결과 분석

### 전체 결과
- 30+ km 구간의 평균 요금이 가장 높으며, \$129.01임.  
- 0-3 km 구간의 평균 요금이 가장 낮으며, \$13.07임.  
- 0-3 km 구간에서 가장 많은 운행이 이루어졌으며, 10,347,228건임.  
- 30+ km 구간에서 가장 적은 운행이 이루어졌으며, 10,866건임.  

### 결론
- **장거리 운행**: 평균 요금이 높고 운행 횟수는 적음.
- **단거리 운행**: 평균 요금이 낮고 운행 횟수는 많음.
- **중간 거리 구간**: 요금과 운행 횟수가 점진적으로 변화함.

### 인사이트
- 운행 거리가 증가함에 따라 요금도 비례적으로 증가하는 경향을 보임.
- 대부분의 운행은 짧은 거리(0-3 km) 구간에서 이루어지며, 그 다음으로는 3-6 km 구간이 많음.
- 장거리 구간에서는 운행 횟수가 상대적으로 적지만, 평균 요금이 훨씬 높음.


# 2. 피크 시간대 요금 분석

In [18]:
# 쿼리
peak_hours_query = '''
SELECT 
  pickup_time AS hour,
  ROUND(AVG(total_amount), 2) AS avg_fare,
  COUNT(total_amount) AS num_trips
FROM comb
GROUP BY pickup_time
ORDER BY hour;
'''
peak_hours_df = spark.sql(peak_hours_query)
peak_hours_df.show(24)

[Stage 17:==================================>                   (129 + 2) / 200]

+----+--------+---------+
|hour|avg_fare|num_trips|
+----+--------+---------+
|   0|   21.03|   252406|
|   1|   20.37|   147940|
|   2|   19.77|    88174|
|   3|   19.95|    50557|
|   4|   22.64|    33862|
|   5|   24.19|    64658|
|   6|   19.63|   210817|
|   7|   17.53|   394091|
|   8|   16.71|   583266|
|   9|   16.58|   650890|
|  10|   16.61|   727994|
|  11|   16.58|   806514|
|  12|    16.7|   893464|
|  13|   16.92|   923919|
|  14|   17.45|  1007120|
|  15|   17.89|  1017848|
|  16|   18.96|   972418|
|  17|   18.65|  1016672|
|  18|    18.1|  1025067|
|  19|   17.94|   865410|
|  20|   18.13|   664606|
|  21|   18.87|   584488|
|  22|   19.56|   513630|
|  23|   20.43|   394938|
+----+--------+---------+



In [19]:
# 데이터프레임

# 피크 시간대별 평균 요금 계산 및 운행 횟수 집계
peak_hours_df = comb_df.groupBy("pickup_time").agg(
    round(avg("total_amount"), 2).alias("avg_fare"),
    count("total_amount").alias("num_trips")
).orderBy("pickup_time")

# 결과 출력
peak_hours_df.show(24)


[Stage 20:=============================================>        (167 + 2) / 200]

+-----------+--------+---------+
|pickup_time|avg_fare|num_trips|
+-----------+--------+---------+
|          0|   21.03|   252406|
|          1|   20.37|   147940|
|          2|   19.77|    88174|
|          3|   19.95|    50557|
|          4|   22.64|    33862|
|          5|   24.19|    64658|
|          6|   19.63|   210817|
|          7|   17.53|   394091|
|          8|   16.71|   583266|
|          9|   16.58|   650890|
|         10|   16.61|   727994|
|         11|   16.58|   806514|
|         12|    16.7|   893464|
|         13|   16.92|   923919|
|         14|   17.45|  1007120|
|         15|   17.89|  1017848|
|         16|   18.96|   972418|
|         17|   18.65|  1016672|
|         18|    18.1|  1025067|
|         19|   17.94|   865410|
|         20|   18.13|   664606|
|         21|   18.87|   584488|
|         22|   19.56|   513630|
|         23|   20.43|   394938|
+-----------+--------+---------+



> 결과 분석

### 전체 결과
- **평균 요금이 가장 높은 시간대**: 5시, 평균 요금 \$24.19임.
- **평균 요금이 가장 낮은 시간대**: 9시, 11시, 평균 요금 \$16.58임.
- **가장 많은 운행이 이루어진 시간대**: 15시, 1,017,848건임.
- **가장 적은 운행이 이루어진 시간대**: 3시, 50,557건임.

### 결론
- **심야 시간대 (0시 - 5시)**: 평균 요금이 높고, 운행 횟수는 적음.
- **출퇴근 시간대 (7시 - 9시, 17시 - 19시)**: 평균 요금이 낮고, 운행 횟수가 많음.
- **일반 시간대 (10시 - 16시)**: 평균 요금과 운행 횟수가 중간 수준임.

### 인사이트
- **심야 시간대**: 택시 요금이 높음. 이 시간대에는 이동 수요가 적어 요금이 더 높아지는 경향이 있음.
- **출퇴근 시간대**: 택시 이용이 활발하고, 운행 횟수가 많아짐. 이 시간대에는 이동 수요가 많아 요금이 비교적 낮음.
- **일반 시간대**: 요금과 운행 횟수가 비교적 균형을 이룸. 이 시간대는 특별히 높거나 낮은 요금 없이 안정적인 운행이 이루어짐.

# 3. 지불 유형별 요금, 팁 분석

In [27]:
# 쿼리
payment_tip_query = '''
SELECT 
  payment_type,
  ROUND(AVG(total_amount), 2) AS avg_fare,
  ROUND(AVG(tip_amount), 2) AS avg_tip
FROM comb
GROUP BY payment_type
ORDER BY payment_type;
'''
payment_tip_df = spark.sql(payment_tip_query)
payment_tip_df.show()

[Stage 52:========================================>             (150 + 2) / 200]

+------------+--------+-------+
|payment_type|avg_fare|avg_tip|
+------------+--------+-------+
|           1|   18.72|   2.88|
|           2|   15.51|    0.0|
|           3|   15.95|    0.0|
|           4|   16.47|    0.0|
+------------+--------+-------+



In [24]:
# 데이터프레임

# 각 지불 유형별 평균 요금과 평균 팁 계산 및 지불 유형 순서대로 정렬
payment_tip_df = comb_df.groupBy("payment_type").agg(
    round(avg("total_amount"), 2).alias("avg_fare"),
    round(avg("tip_amount"), 2).alias("avg_tip")
).orderBy("payment_type")

# 결과 출력
payment_tip_df.select("payment_type", "avg_fare", "avg_tip").show()


[Stage 43:=================================================>    (185 + 2) / 200]

+------------+--------+-------+
|payment_type|avg_fare|avg_tip|
+------------+--------+-------+
|           1|   18.72|   2.88|
|           2|   15.51|    0.0|
|           3|   15.95|    0.0|
|           4|   16.47|    0.0|
+------------+--------+-------+



> 결과 분석

### 전체 결과
- **Payment Type 1**: 평균 요금 \$18.72, 평균 팁 \$2.88  
- **Payment Type 2**: 평균 요금 \$15.51, 평균 팁 \$0.00  
- **Payment Type 3**: 평균 요금 \$15.95, 평균 팁 \$0.00  
- **Payment Type 4**: 평균 요금 \$16.47, 평균 팁 \$0.00  

### 결론
- **Payment Type 1**: 다른 지불 유형에 비해 평균 요금과 평균 팁이 가장 높음.
- **Payment Type 2, 3, 4**: 평균 요금이 비슷하며, 팁이 없음.

### 인사이트
- **Payment Type 1**: 평균 요금이 높고 팁도 높음.
- **Payment Type 2, 3, 4**: 팁이 없고 평균 요금이 상대적으로 낮음.


# 4. 승차지역 / 하차지역별 평균거리, 요금

In [30]:
# 쿼리
distance_fare_query = '''
SELECT 
  pickup_zone,
  dropoff_zone,
  ROUND(AVG(trip_distance), 2) AS avg_distance,
  ROUND(AVG(total_amount), 2) AS avg_fare
FROM comb
GROUP BY pickup_zone, dropoff_zone
ORDER BY avg_distance DESC;
'''
distance_fare_df = spark.sql(distance_fare_query)

In [31]:
# 데이터프레임

# 각 승차지역 및 하차지역별 평균 거리와 평균 요금을 계산하고, 소숫점 2자리까지 표시
distance_fare_df = comb_df.groupBy("pickup_zone", "dropoff_zone").agg(
    round(avg("trip_distance"), 2).alias("avg_distance"),
    round(avg("total_amount"), 2).alias("avg_fare")
).orderBy("avg_distance", ascending=False)

In [32]:
# 평균 거리 기준 상위 10개 지역
distance_fare_df.orderBy(col("avg_distance").desc()).show(10)

# 평균 요금 기준 상위 10개 지역
distance_fare_df.orderBy(col("avg_distance").desc()).show(10)

# 평균 거리 기준 하위 10개 지역
distance_fare_df.orderBy(col("avg_distance").asc()).show(10)

# 평균 요금 기준 하위 10개 지역
distance_fare_df.orderBy(col("avg_fare").asc()).show(10)

# 요약 통계
distance_fare_df.describe(["avg_distance", "avg_fare"]).show()

+--------------------+--------------------+------------+--------+
|         pickup_zone|        dropoff_zone|avg_distance|avg_fare|
+--------------------+--------------------+------------+--------+
|East New York/Pen...|                  NA|       63.29|  255.28|
|Heartland Village...|                  NA|       56.87|   208.9|
|      Sheepshead Bay|            Longwood|       54.12|   93.85|
|         Eastchester|Charleston/Totten...|       51.37|   156.4|
|   Rossville/Woodrow|     Pelham Bay Park|       51.28|   151.0|
|Charleston/Totten...|     Pelham Bay Park|       51.02|   157.5|
|Charleston/Totten...|         Eastchester|       49.48|  149.26|
|          Co-Op City|Charleston/Totten...|       48.87|  149.21|
|          Co-Op City|     Mariners Harbor|       48.75|  157.99|
|         Eastchester|   Rossville/Woodrow|       48.66|  138.04|
+--------------------+--------------------+------------+--------+
only showing top 10 rows



+--------------------+--------------------+------------+--------+
|         pickup_zone|        dropoff_zone|avg_distance|avg_fare|
+--------------------+--------------------+------------+--------+
|East New York/Pen...|                  NA|       63.29|  255.28|
|Heartland Village...|                  NA|       56.87|   208.9|
|      Sheepshead Bay|            Longwood|       54.12|   93.85|
|         Eastchester|Charleston/Totten...|       51.37|   156.4|
|   Rossville/Woodrow|     Pelham Bay Park|       51.28|   151.0|
|Charleston/Totten...|     Pelham Bay Park|       51.02|   157.5|
|Charleston/Totten...|         Eastchester|       49.48|  149.26|
|          Co-Op City|Charleston/Totten...|       48.87|  149.21|
|          Co-Op City|     Mariners Harbor|       48.75|  157.99|
|         Eastchester|   Rossville/Woodrow|       48.66|  138.04|
+--------------------+--------------------+------------+--------+
only showing top 10 rows



+--------------------+-----------------+------------+--------+
|         pickup_zone|     dropoff_zone|avg_distance|avg_fare|
+--------------------+-----------------+------------+--------+
|Forest Park/Highl...|      Kew Gardens|        0.04|     3.3|
|  Van Cortlandt Park|               NV|        0.04|     0.9|
|       East New York|               NV|        0.05|     4.3|
|           Bay Ridge|               NV|        0.05|     3.8|
|  Claremont/Bathgate|     Crotona Park|        0.06|     4.3|
|    Sunset Park West|               NV|        0.06|     4.3|
|        Astoria Park|     Astoria Park|         0.1|     3.3|
|        East Tremont|               NV|         0.1|    3.47|
|   Rossville/Woodrow|Rossville/Woodrow|         0.1|     0.3|
|            Rosedale|  Cambria Heights|         0.1|     3.3|
+--------------------+-----------------+------------+--------+
only showing top 10 rows



+-----------------+--------------------+------------+--------+
|      pickup_zone|        dropoff_zone|avg_distance|avg_fare|
+-----------------+--------------------+------------+--------+
|        Flatlands|     Lenox Hill East|        0.13|     0.0|
|     Bloomingdale|        Coney Island|       20.43|     0.0|
|Rossville/Woodrow|   Rossville/Woodrow|         0.1|     0.3|
|          Jamaica|  South Williamsburg|        12.0|     0.3|
|      Westerleigh|Bloomfield/Emerso...|         1.2|     0.3|
|    South Jamaica|         Kew Gardens|         1.9|     0.3|
|          Jamaica|       Willets Point|         4.6|     0.3|
|       Bronx Park|  Woodlawn/Wakefield|         2.6|     0.3|
|      Parkchester|        Baisley Park|       14.36|    0.31|
|    Fresh Meadows|         JFK Airport|         1.0|    0.31|
+-----------------+--------------------+------------+--------+
only showing top 10 rows



+-------+------------------+-----------------+
|summary|      avg_distance|         avg_fare|
+-------+------------------+-----------------+
|  count|             29959|            29959|
|   mean|10.148192863580233|41.46117927834707|
| stddev| 6.786928575677103|23.87105460877794|
|    min|              0.04|              0.0|
|    max|             63.29|           1165.3|
+-------+------------------+-----------------+



> 결과 분석

### 전체 결과
- **평균 거리 기준 상위 10개 지역**:
  - East New York/Pennsylvania와 Heartland Village 지역은 평균 거리가 50km 이상으로 가장 길며, 요금도 상대적으로 높음.
  - 대부분의 상위 지역들은 장거리 운행에 해당하며, 평균 요금도 높음.

- **평균 요금 기준 상위 10개 지역**:
  - 평균 거리 기준 상위 10개 지역과 일치하는 경우가 많음.
  - East New York/Pennsylvania와 Heartland Village는 높은 평균 요금이 두드러짐.

- **평균 거리 기준 하위 10개 지역**:
  - Forest Park/Highland Park와 Van Cortlandt Park는 평균 거리가 0.04km로 가장 짧음.
  - 대부분의 하위 지역들은 매우 짧은 거리를 운행하는 경우가 많음.

- **평균 요금 기준 하위 10개 지역**:
  - Flatlands와 Bloomingdale은 평균 요금이 $0.00로 나타남.
  - 대부분의 하위 지역들은 평균 요금이 매우 낮거나 무료인 경우가 있음.

- **요약 통계**:
  - 평균 거리는 약 10.15km, 평균 요금은 약 $41.46으로 나타남.
  - 최대 거리는 63.29km, 최대 요금은 $1165.30으로 나타남.

### 결론
- 장거리 운행: 평균 요금이 높고 운행 횟수는 적음.
- 단거리 운행: 평균 요금이 낮고 운행 횟수는 많음.
- 중간 거리 구간: 요금과 운행 횟수가 점진적으로 변화함.

### 인사이트
- 장거리 운행에서는 요금이 높아지는 경향이 뚜렷함.
- 단거리 운행에서는 요금이 상대적으로 낮고 운행 횟수가 많음.
- 중간 거리 구간에서는 요금과 운행 횟수가 균형을 이루는 경향을 보임.
- 운행 거리와 요금은 대부분의 경우 비례하는 경향이 있으며, 이는 택시 요금 구조와 관련이 있음.


# 5. 팁의 비율에 따른 거리, 여행 건수 서비스 관련 분석

In [33]:
# 쿼리
tip_ratio_query = '''
SELECT 
  CASE 
    WHEN tip_amount / total_amount < 0.1 THEN '0-10%'
    WHEN tip_amount / total_amount >= 0.1 AND tip_amount / total_amount < 0.2 THEN '10-20%'
    WHEN tip_amount / total_amount >= 0.2 AND tip_amount / total_amount < 0.3 THEN '20-30%'
    WHEN tip_amount / total_amount >= 0.3 AND tip_amount / total_amount < 0.4 THEN '30-40%'
    ELSE '40%+'
  END AS tip_ratio,
  ROUND(AVG(trip_distance), 2) AS avg_distance,
  COUNT(*) AS num_trips
FROM comb
GROUP BY tip_ratio
ORDER BY tip_ratio;
'''
tip_ratio_df = spark.sql(tip_ratio_query)
tip_ratio_df.show()

[Stage 76:==================================================>     (10 + 1) / 11]

+---------+------------+---------+
|tip_ratio|avg_distance|num_trips|
+---------+------------+---------+
|    0-10%|        3.09|  4971107|
|   10-20%|        2.84|  7339171|
|   20-30%|        2.48|  1511876|
|   30-40%|         2.0|    46236|
|     40%+|        2.28|    22359|
+---------+------------+---------+



In [36]:
# 데이터프레임

# 팁 비율 구간별로 평균 거리와 여행 건수를 계산
tip_ratio_df = comb_df.withColumn(
    "tip_ratio", 
    when((col("tip_amount") / col("total_amount")) < 0.1, '0-10%')
    .when((col("tip_amount") / col("total_amount")).between(0.1, 0.2), '10-20%')
    .when((col("tip_amount") / col("total_amount")).between(0.2, 0.3), '20-30%')
    .when((col("tip_amount") / col("total_amount")).between(0.3, 0.4), '30-40%')
    .otherwise('40%+')
).groupBy("tip_ratio").agg(
    round(avg("trip_distance"), 2).alias("avg_distance"),
    count("*").alias("num_trips")
).orderBy("tip_ratio")

# 결과 출력
tip_ratio_df.show()

[Stage 82:==================================================>     (10 + 1) / 11]

+---------+------------+---------+
|tip_ratio|avg_distance|num_trips|
+---------+------------+---------+
|    0-10%|        3.09|  4971107|
|   10-20%|        2.81|  7839352|
|   20-30%|        2.57|  1011837|
|   30-40%|         2.0|    46126|
|     40%+|        2.28|    22327|
+---------+------------+---------+



> 결과 분석
### 전체 결과
- **0-10% 팁 비율**:
  - 평균 거리: 3.09 km
  - 여행 건수: 4,971,107건
- **10-20% 팁 비율**:
  - 평균 거리: 2.81 km
  - 여행 건수: 7,839,352건
- **20-30% 팁 비율**:
  - 평균 거리: 2.57 km
  - 여행 건수: 1,011,837건
- **30-40% 팁 비율**:
  - 평균 거리: 2.00 km
  - 여행 건수: 46,126건
- **40%+ 팁 비율**:
  - 평균 거리: 2.28 km
  - 여행 건수: 22,327건

### 결론
- **팁 비율이 낮을수록 여행 거리가 길다**: 0-10% 팁 비율 구간에서 평균 거리가 가장 길며, 팁 비율이 높아질수록 평균 거리가 짧아짐.
- **팁 비율이 높을수록 여행 건수는 적다**: 10-20% 팁 비율 구간에서 가장 많은 여행 건수가 발생하였으며, 팁 비율이 높아질수록 여행 건수는 줄어듦.

### 인사이트
- **0-10% 팁 비율**: 이 구간에서는 여행 거리가 길고, 많은 여행이 이루어짐. 이는 상대적으로 긴 거리를 이동하는 승객들이 팁을 적게 주는 경향이 있음을 시사함.
- **10-20% 팁 비율**: 이 구간에서 가장 많은 여행 건수가 발생함. 평균 거리가 약간 짧아지지만 여전히 많은 승객들이 이 범위 내에서 팁을 줌.
- **20-30% 팁 비율**: 이 구간에서는 여행 거리와 건수가 더 감소함. 팁을 적당히 많이 주는 승객들의 경우, 이동 거리가 더 짧아지는 경향이 있음.
- **30-40% 팁 비율**: 이 구간에서는 평균 거리가 더 짧아지고, 여행 건수도 급격히 줄어듦. 매우 높은 팁 비율을 주는 승객들은 주로 짧은 거리를 이동함.
- **40%+ 팁 비율**: 이 구간에서는 여행 거리가 다시 약간 길어지지만, 여행 건수는 매우 적음. 극도로 높은 팁을 주는 승객들은 특별한 경우에 한정될 가능성이 큼.

In [37]:
spark.stop()